<div style="background-color:	#006400; text-align:center; vertical-align: middle; padding:40px 0; margin-top:30px">
<h1 style="color:white">DataLab - Sprint 2</h1>
<b style="color:white">Jasper Duncker, Luca Oldenbeuving, Mitchel Wouters & Ifraah Ghedi</b>
    </div>

<h2 style="color:#32CD32">1. Voorbereiding</h2>

In [1]:
##importeren van benodigdheden

import numpy as np # linear algebra
import pandas as pd # data processing
import sqlite3 #sql
pd.set_option('display.max_columns', 200)
database ='database.sqlite'


In [2]:
#zien welke tabellen er beschikbaar zijn

conn = sqlite3.connect(database)

tables = pd.read_sql("""SELECT *
                        FROM sqlite_master
                        WHERE type='table'
                        """, conn)
tables

,type,name,tbl_name,rootpage,sql
0,table,sqlite_sequence,sqlite_sequence,4,"CREATE TABLE sqlite_sequence(name,seq)"
1,table,Player_Attributes,Player_Attributes,11,"CREATE TABLE ""Player_Attributes"" (\n\t`id`\tIN..."
2,table,Player,Player,14,CREATE TABLE `Player` (\n\t`id`\tINTEGER PRIMA...
3,table,Match,Match,18,CREATE TABLE `Match` (\n\t`id`\tINTEGER PRIMAR...
4,table,League,League,24,CREATE TABLE `League` (\n\t`id`\tINTEGER PRIMA...
5,table,Country,Country,26,CREATE TABLE `Country` (\n\t`id`\tINTEGER PRIM...
6,table,Team,Team,29,"CREATE TABLE ""Team"" (\n\t`id`\tINTEGER PRIMARY..."
7,table,Team_Attributes,Team_Attributes,2,CREATE TABLE `Team_Attributes` (\n\t`id`\tINTE...


<h2 style="color:#32CD32">2. Opdracht 1</h2>


<b>Beschrijf welke tabbelen beschikbaar zijn</b>

In [3]:
#aparte tabelen maken
with sqlite3.connect('database.sqlite') as con:
    Country = pd.read_sql_query("SELECT * from Country", con)
    Match = pd.read_sql_query("SELECT * from Match", con)
    League = pd.read_sql_query("SELECT * from League", con)
    Team = pd.read_sql_query("SELECT * from Team", con)
    Player_Attributes = pd.read_sql_query("SELECT * from Player_Attributes", con)
    Player = pd.read_sql_query("SELECT * from Team", con)
    sqlite_sequence = pd.read_sql_query("SELECT * from sqlite_sequence", con)
    Team_Attributes = pd.read_sql_query("SELECT * from Team_Attributes", con)

De beschikbare tabellen zijn country, match, league, team, plater_atributes, player, sqlite_sequence & Team_Attributes

<b>Beschrijf de data die je kunt vinden in iedere tabel. </b>

In [4]:
for table in tables['name']:
    schema = pd.read_sql(f"PRAGMA table_info('{table}')", conn)
    print(f"\n\nTable: {table}")
    print(schema)



Table: sqlite_sequence
   cid  name type  notnull dflt_value  pk
0    0  name             0       None   0
1    1   seq             0       None   0


Table: Player_Attributes
    cid                 name     type  notnull dflt_value  pk
0     0                   id  INTEGER        0       None   1
1     1   player_fifa_api_id  INTEGER        0       None   0
2     2        player_api_id  INTEGER        0       None   0
3     3                 date     TEXT        0       None   0
4     4       overall_rating  INTEGER        0       None   0
5     5            potential  INTEGER        0       None   0
6     6       preferred_foot     TEXT        0       None   0
7     7  attacking_work_rate     TEXT        0       None   0
8     8  defensive_work_rate     TEXT        0       None   0
9     9             crossing  INTEGER        0       None   0
10   10            finishing  INTEGER        0       None   0
11   11     heading_accuracy  INTEGER        0       None   0
12   12        s

<b>Bepaal de volgende gegevens van jouw club:</b>
    
    -> team naam
    -> team id
    -> competitie id
    -> competitie naam
    -> De gegevens van alle wedstrijden die jouw club heeft gespeeld.

In [5]:
# Connect to the SQLite database
conn = sqlite3.connect(database)

# Query de team informatie
team_query = "SELECT team_long_name, team_short_name, team_api_id, \
                    league.name AS league_name, match.league_id AS league_id \
              FROM team \
              JOIN match ON team.team_api_id = match.home_team_api_id OR team.team_api_id = match.away_team_api_id \
              JOIN league ON match.league_id = league.id \
              WHERE team_long_name LIKE '%Ajax%'"
team_df = pd.read_sql(team_query, conn)

# Query van de wedstrijden die ajax heeft gespeeld 
match_query = "SELECT * \
               FROM match \
               WHERE home_team_api_id = %s OR away_team_api_id = %s" % (team_df['team_api_id'][0], team_df['team_api_id'][0])
match_df = pd.read_sql(match_query, conn)

# Zet match_df om in DataFrame
match_df = pd.DataFrame(match_df)

# Print de team informatie
print("Team Name: ", team_df['team_long_name'][0])
print("Team ID: ", team_df['team_api_id'][0])
print("League ID: ", team_df['league_id'][0])
print("Competition Name: ", team_df['league_name'][0])

# Weergeef alle details van de wedstrijden die Ajax heeft gespeeld
display("Matches Played: ", len(match_df))
display(match_df.head())


Team Name:  Ajax
Team ID:  8593
League ID:  13274
Competition Name:  Netherlands Eredivisie


'Matches Played: '

272

,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,home_player_X1,home_player_X2,home_player_X3,home_player_X4,home_player_X5,home_player_X6,home_player_X7,home_player_X8,home_player_X9,home_player_X10,home_player_X11,away_player_X1,away_player_X2,away_player_X3,away_player_X4,away_player_X5,away_player_X6,away_player_X7,away_player_X8,away_player_X9,away_player_X10,away_player_X11,home_player_Y1,home_player_Y2,home_player_Y3,home_player_Y4,home_player_Y5,home_player_Y6,home_player_Y7,home_player_Y8,home_player_Y9,home_player_Y10,home_player_Y11,away_player_Y1,away_player_Y2,away_player_Y3,away_player_Y4,away_player_Y5,away_player_Y6,away_player_Y7,away_player_Y8,away_player_Y9,away_player_Y10,away_player_Y11,home_player_1,home_player_2,home_player_3,home_player_4,home_player_5,home_player_6,home_player_7,home_player_8,home_player_9,home_player_10,home_player_11,away_player_1,away_player_2,away_player_3,away_player_4,away_player_5,away_player_6,away_player_7,away_player_8,away_player_9,away_player_10,away_player_11,goal,shoton,shotoff,foulcommit,card,cross,corner,possession,B365H,B365D,B365A,BWH,BWD,BWA,IWH,IWD,IWA,LBH,LBD,LBA,PSH,PSD,PSA,WHH,WHD,WHA,SJH,SJD,SJA,VCH,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA
0,13276,13274,13274,2008/2009,1,2008-08-30 00:00:00,508304,8525,8593,2,1,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,42710.0,45488.0,26637.0,45485.0,NaN,27367.0,33684.0,NaN,33685.0,45830.0,26519.0,30841.0,26468.0,37643.0,26005.0,37762.0,33683.0,26459.0,30603.0,40636.0,36784.0,45174.0,None,None,None,None,None,None,None,None,5.50,3.60,1.57,5.75,3.75,1.50,5.00,3.7,1.5,5.00,3.50,1.57,NaN,NaN,NaN,5.50,3.6,1.5,6.50,4.00,1.50,5.00,3.6,1.55,5.00,3.7,1.60,5.50,3.6,1.53
1,13288,13274,13274,2008/2009,10,2008-11-09 00:00:00,523736,8593,8614,5,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,1.17,7.50,13.00,1.20,5.75,10.25,1.15,6.0,12.0,1.20,5.00,11.00,NaN,NaN,NaN,1.17,5.5,12.0,1.20,5.75,15.00,1.17,6.0,11.00,1.20,6.0,11.00,1.17,6.0,13.00
2,13297,13274,13274,2008/2009,11,2008-11-16 00:00:00,523745,8593,8640,4,1,1.0,2.0,4.0,6.0,8.0,3.0,5.0,7.0,3.0,5.0,7.0,1.0,2.0,4.0,6.0,8.0,2.0,4.0,6.0,8.0,4.0,6.0,1.0,3.0,3.0,3.0,3.0,7.0,7.0,7.0,10.0,10.0,10.0,1.0,3.0,3.0,3.0,3.0,7.0,7.0,7.0,7.0,10.0,10.0,26002.0,109651.0,37643.0,26005.0,37762.0,118730.0,26535.0,26459.0,23951.0,40636.0,30603.0,36617.0,41165.0,30964.0,34479.0,37194.0,39359.0,5703.0,26440.0,145586.0,26434.0,26451.0,<goal><value><comment>n</comment><stats><goals...,<shoton><value><stats><shoton>1</shoton></stat...,<shotoff><value><stats><shotoff>1</shotoff></s...,<foulcommit><value><stats><foulscommitted>1</f...,<card><value><comment>y</comment><stats><ycard...,<cross><value><stats><crosses>1</crosses></sta...,<corner><value><stats><corners>1</corners></st...,<possession />,2.20,3.25,3.00,2.15,3.20,3.10,2.00,3.2,3.2,2.10,3.20,3.00,NaN,NaN,NaN,2.05,3.3,3.0,2.25,3.40,2.75,2.10,3.3,3.00,2.10,3.3,3.10,2.10,3.3,3.10
3,13306,13274,13274,2008/2009,12,2008-11-23 00:00:00,523754,10229,8593,2,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,2.30,3.30,2.75,2.40,3.30,2.60,2.30,3.2,2.5,2.38,3.20,2.60,NaN,NaN,NaN,2.25,3.3,2.8,2.30,3.40,2.88,2.35,3.3,2.60,2.35,3.3,2.75,2.25,3.3,2.80
4,13316,13274,13274,2008/2009,13,2008-11-30 00:00:00,523764,8593,9908,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

<h2 style="color:#32CD32">2. Opdracht 2</h2>

In [6]:
## functie van een show query, zodat wij die niet telkens hoeven te schrijven.
def show_query(query):
    """functie van een show query, zodat wij die niet telkens hoeven te schrijven.
    Parameters
   ----------
    query : str
        Een docstring waar je een sql query in schrijft.
    
    Returns
    -------
    pandas.DataFrame
        Het resultaat van de query in een DataFrame.

    """
    result = pd.read_sql(query, conn)
    return result

<b>Maak een functie waarmee je per seizoen voor een gekozen club o.a. het volgende kunt bepalen:</b>
        
        
    -> Aantal wedstrijden in dat seizoen.
    -> Aantal punten behaald in dat seizoen.
    -> Gemiddeld aantal punten per wedstrijd.
    -> Winst-, verlies- en gelijk percentage in uitwedstrijden en thuiswedstrijden.


In [7]:
# Aantal wedstrijden in dat seizoen.
def Seizoens_info(seizoen,club=8593):
    '''Neemt seizoen en clubnaam en maakt een DataFrame met informatie over gespeelde wedstrijden en behaalde punten.
   
   Parameters
   ----------
    seizoen : str
        Het gekozen seizoenwaar je informatie over wilt opzoeken.
    club : int, default = 8593
        De gekozen club_id waar je informatie over wilt opzoeken.

    Returns
    -------
    df: pandas.DataFrame
        De Dataframe met de kollommen: 
    aantal_matches: int
    punten_behaald: int 
    punten_gemiddeld : int
    Home_winst_percentage : int
    Home_draws_percentage : int:
    Home_lose_percentage: int
    away_win_percentage: int
    away_draws_percentage: int  
    away_lose_percentage: int
    club_id : int
        '''
    
    query2 = f"""
        SELECT 
        COUNT(CASE WHEN home_team_api_id = {club} AND home_team_goal > away_team_goal THEN 1 END) AS home_wins,
        COUNT(CASE WHEN home_team_api_id = {club} AND home_team_goal = away_team_goal THEN 1 END) AS home_draws,
        COUNT(CASE WHEN home_team_api_id = {club} AND home_team_goal < away_team_goal THEN 1 END) AS home_loses,
        COUNT(CASE WHEN home_team_api_id = {club} THEN 1 END) as home_games,

        COUNT(CASE WHEN away_team_api_id = {club} AND away_team_goal > home_team_goal THEN 1 END) AS away_wins,
        COUNT(CASE WHEN away_team_api_id = {club} AND away_team_goal = home_team_goal THEN 1 END) AS away_draws,
        COUNT(CASE WHEN away_team_api_id = {club} AND away_team_goal = home_team_goal THEN 1 END) AS away_loses,
        COUNT(CASE WHEN away_team_api_id = {club} THEN 1 END) as away_games
    FROM match
    WHERE season LIKE '{seizoen}'
        AND (home_team_api_id = {club}
        OR away_team_api_id = {club})"""
    
    df = pd.DataFrame(show_query(query2))
    data=pd.DataFrame()
    
  
    data['aantal_matches'] = df['home_games'] + df['away_games']
    data['punten_behaald'] = df['home_wins']*3 + df['home_draws'] + df['away_wins']*3 + df['away_draws']
    data['punten_gemiddeld'] = data['punten_behaald']/(df['home_games'] + df['away_games'])

    data['Home_winst_percentage'] = round(df['home_wins']/df['home_games']*100 ,2)
    data['Home_draws_percentage'] = round(df['home_draws']/df['home_games']*100,2)
    data['Home_lose_percentage'] = round(df['home_loses']/df['home_games']*100,2)
    
    data['away_win_percentage'] = round(df['away_wins']/df['away_games']*100,2)
    data['away_draws_percentage'] = round(df['away_draws']/df['away_games']*100,2)
    data['away_lose_percentage'] = round(df['away_loses']/df['away_games']*100,2)
    
    data['club_id'] = club

    
    return data


In [8]:
Seizoens_info('2015/2016').stack()

0  aantal_matches             34.000000
   punten_behaald             82.000000
   punten_gemiddeld            2.411765
   Home_winst_percentage      76.470000
   Home_draws_percentage      17.650000
   Home_lose_percentage        5.880000
   away_win_percentage        70.590000
   away_draws_percentage      23.530000
   away_lose_percentage       23.530000
   club_id                  8593.000000
dtype: float64

### Ranglijst

In [9]:
query="""
SELECT DISTINCT(home_team_api_id)
FROM match
WHERE country_id = 13274;
"""
eredivisie_clubs = pd.DataFrame(show_query(query))

kaas = []

for i,r in eredivisie_clubs.iterrows():
    kaas.append(Seizoens_info('2015/2016',r['home_team_api_id']))

big_df = pd.concat(kaas, ignore_index=True)


In [10]:
big_df.sort_values('punten_behaald',ascending=False)

,aantal_matches,punten_behaald,punten_gemiddeld,Home_winst_percentage,Home_draws_percentage,Home_lose_percentage,away_win_percentage,away_draws_percentage,away_lose_percentage,club_id
17,34,84,2.470588,76.47,17.65,5.88,76.47,17.65,17.65,8640
10,34,82,2.411765,76.47,17.65,5.88,70.59,23.53,23.53,8593
12,34,63,1.852941,64.71,17.65,17.65,47.06,17.65,17.65,10235
8,34,59,1.735294,58.82,11.76,29.41,47.06,17.65,17.65,10229
4,34,53,1.558824,52.94,23.53,23.53,35.29,23.53,23.53,9908
5,34,51,1.500000,52.94,29.41,17.65,29.41,23.53,23.53,9791
11,34,50,1.470588,58.82,17.65,23.53,23.53,29.41,29.41,8674
21,34,48,1.411765,58.82,11.76,29.41,23.53,23.53,23.53,6413
0,34,46,1.352941,41.18,23.53,35.29,29.41,35.29,35.29,8277
3,34,46,1.352941,64.71,11.76,23.53,11.76,29.41,29.41,8464


In [11]:
team_id = ("""
SELECT  team_api_id, team_long_name
FROM team
""")

show_query(team_id)


team_id_df = pd.DataFrame(show_query(team_id))

team_id_df


ranglijst= team_id_df.merge(big_df, left_on='team_api_id', right_on='club_id').sort_values('punten_behaald', ascending=False).set_index('team_api_id').dropna()
ranglijst


,team_long_name,aantal_matches,punten_behaald,punten_gemiddeld,Home_winst_percentage,Home_draws_percentage,Home_lose_percentage,away_win_percentage,away_draws_percentage,away_lose_percentage,club_id
team_api_id,,,,,,,,,,,
8640,PSV,34,84,2.470588,76.47,17.65,5.88,76.47,17.65,17.65,8640
8593,Ajax,34,82,2.411765,76.47,17.65,5.88,70.59,23.53,23.53,8593
10235,Feyenoord,34,63,1.852941,64.71,17.65,17.65,47.06,17.65,17.65,10235
10229,AZ,34,59,1.735294,58.82,11.76,29.41,47.06,17.65,17.65,10229
9908,FC Utrecht,34,53,1.558824,52.94,23.53,23.53,35.29,23.53,23.53,9908
9791,Heracles Almelo,34,51,1.500000,52.94,29.41,17.65,29.41,23.53,23.53,9791
8674,FC Groningen,34,50,1.470588,58.82,17.65,23.53,23.53,29.41,29.41,8674
6413,PEC Zwolle,34,48,1.411765,58.82,11.76,29.41,23.53,23.53,23.53,6413
8464,N.E.C.,34,46,1.352941,64.71,11.76,23.53,11.76,29.41,29.41,8464
